<a href="https://colab.research.google.com/github/nhatpham2016/Alpaca-Trading-API-Guide-A-Step-by-step-Guide/blob/master/sales_strategies_reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package

In [140]:
!pip3 install youtube-search-python -q
!pip install youtube-transcript-api -q
!pip install pytube -q
!pip install -U --quiet  langgraph langchain-fireworks
!pip install -U --quiet tavily-python
!pip install langchain -q
!pip install python-dotenv -q
!pip install -U langchain-community -q


In [141]:
import getpass
import os
from google.colab import userdata
userdata.get('TAVILY_API_KEY')
userdata.get('FIREWORKS_API_KEY')

def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = userdata.get(var)  #getpass.getpass(var)

_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("FIREWORKS_API_KEY")

In [142]:
from langchain_fireworks import ChatFireworks
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=32768
)

In [143]:
llm.invoke("why you run too slow?")

AIMessage(content="I'm sorry if it seems like I'm running slow. I'm here to provide helpful, respectful, and positive responses. The speed at which my responses are generated is not a measure of my ability to assist you. I'm here to help answer your questions to the best of my ability. If you have any questions on a specific topic, please feel free to ask!", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 59, 'total_tokens': 140, 'completion_tokens': 81}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-57bfb87c-7218-46e6-9659-1e83bedcbf07-0', usage_metadata={'input_tokens': 59, 'output_tokens': 81, 'total_tokens': 140})

In [144]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [145]:
tool.invoke({"query": "What is the status of Japan economy sept 2024?"})

[{'url': 'https://www.usnews.com/news/business/articles/2024-09-08/japans-economy-is-growing-but-political-uncertainty-is-among-the-risks',
  'content': "Japan's economy grew at an annual rate of 2.9%, slower than the earlier report for 3.1% growth, in the April-June period, boosted by better wages and spending By Associated Press Sept. 8, 2024"},
 {'url': 'https://www.nippon.com/en/in-depth/d00963/',
  'content': 'The current increase of prices is characterized by companies passing through to selling prices the increase of raw material costs accompanying the depreciation of the yen and the ascent of crude oil prices. While the growth rate of real wages will continue to be negative, it is highly probable that this rate will turn positive in the second half of 2024 as nominal wages accelerate their growth and as the rise of prices stabilizes.\n The wage increase rate of the spring wage offensive of 2024 is expected to surpass the increase rate of 2023, and it is highly probable that ser

In [146]:
# -*- coding: utf-8 -*-
"""SalesGPT agent refactored using a reflection-based design with satisfaction check."""

import os
import re
from typing import Any, Dict, List, Union, Callable
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

from langchain.agents import LLMSingleActionAgent, AgentExecutor, Tool
from langchain.agents.agent import AgentOutputParser
from langchain.chains import LLMChain, RetrievalQA
from langchain.llms import BaseLLM
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from typing import Annotated  # Add this import for Annotated support

from langchain.schema import AgentAction, AgentFinish, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

import asyncio


import os
import asyncio
from typing import List, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain_fireworks import ChatFireworks


# Initialize the LLM
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=32768
)

# Utility function to check if an event loop is running
def is_event_loop_running() -> bool:
    """Check if there's an event loop running."""
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False


# Step 1: SalesGPT Chains (Stage Analyzer and Sales Conversation)
class StageAnalyzerChain(LLMChain):
    """Chain to analyze which conversation stage the conversation should move into."""

    @classmethod
    def from_llm(cls, llm, verbose: bool = True) -> LLMChain:
        stage_analyzer_prompt = """You are a sales assistant helping your sales agent determine which stage of a sales conversation should the agent move to, or stay at.
        Following '===' is the conversation history.
        Use this conversation history to make your decision.
        ===
        {conversation_history}
        ===

        Now determine what should be the next immediate conversation stage for the agent in the sales conversation by selecting only from the following options:
        1. Introduction
        2. Qualification
        3. Value proposition
        4. Needs analysis
        5. Solution presentation
        6. Objection handling
        7. Close

        Only answer with a number between 1 and 7."""

        prompt = PromptTemplate(
            template=stage_analyzer_prompt,
            input_variables=["conversation_history"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)


class SalesConversationChain(LLMChain):
    """Chain to generate the next utterance for the conversation."""

    @classmethod
    def from_llm(cls, llm, verbose: bool = True) -> LLMChain:
        sales_agent_prompt = """Your name is {salesperson_name}, and you work at {company_name}. Your business is {company_business}.
        Respond according to the previous conversation history and the stage of the conversation. Only generate one response at a time.
        Conversation history:
        {conversation_history}
        {salesperson_name}:"""

        prompt = PromptTemplate(
            template=sales_agent_prompt,
            input_variables=[
                "salesperson_name",
                "company_name",
                "company_business",
                "conversation_history",
            ],
        )
        print(prompt)
        return cls(prompt=prompt, llm=llm, verbose=verbose)


# # Step 2: Sales Agent State

# Sales Agent State as a TypedDict
class SalesState(TypedDict):
    messages: Annotated[List[str], add_messages]
    staffing_reflection : Annotated[List[str], add_messages]
    macro_data: Annotated[List[str], add_messages]
    region_analysis_data: Annotated[List[str], add_messages]
    customer_segment_reflection: Annotated[List[str], add_messages]
    conversation_stage: Annotated[str, add_messages]
    company_business: Annotated[str, add_messages]
    company_name: Annotated[str, add_messages]
    salesperson_name: Annotated[str, add_messages]
    conversation_history: Annotated[List[str], add_messages]

# Step 3: Nodes for Graph-based Design
async def generation_node(state: SalesState) -> SalesState:
    """Node to generate next utterance in the sales conversation."""
    # No need for 'await' here as 'run' is not asynchronous
    ai_message = state.sales_conversation_utterance_chain.run(
        salesperson_name=state.salesperson_name,
        company_name=state.company_name,
        company_business=state.company_business,
        conversation_history="\n".join([msg.content for msg in state['messages']]),
    )
    # Add the response to the conversation history
    ai_message = f"{state.salesperson_name}: {ai_message}"
    state['messages'].append(AIMessage(content=ai_message))
    return state


# Example async function for reflection node focused on sales strategy evaluation
async def reflection_node(state: SalesState) -> SalesState:
    """Node to evaluate the sales strategies used in the conversation and provide feedback."""

    # Create a reflection prompt to evaluate the sales strategy
    reflection_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You are an expert sales strategist. Evaluate the effectiveness of the current sales strategies used in this conversation.
            Assess if the salesperson’s approach is convincing, clear, and aligns with the customer’s needs.
            Provide feedback on whether the strategy should continue or change, and if the strategy likely leads to a successful outcome.
            At the end of your feedback, include a recommendation such as: 'The strategy is sound, awaiting customer response' or 'Consider changing the approach to address customer concerns better'."""),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    # Bind the state messages to the reflection prompt
    reflect = reflection_prompt | llm

    # Generate a reflection based on the sales strategies and current conversation state
    reflection = await reflect.ainvoke(state['messages'])

    # Output the reflection for debugging or logging purposes
    print("Reflection:", reflection.content)

    # Append the reflection as an AIMessage to the conversation history
    state['messages'].append(AIMessage(content=reflection.content))

    # Return the updated state for further decision-making
    return state

async def satisfaction_check_node(state: SalesState) -> SalesState:
    """Node to check if the user is satisfied and ready to end the conversation."""
    latest_message = state['messages'][-1].content.lower()

    # Check if the latest message contains satisfaction markers (e.g., order placed, problem solved)
    if "place an order" in latest_message or "i'm satisfied" in latest_message or "end the conversation" in latest_message:
        print("User appears satisfied. Ending the conversation.")
        return END

    return state  # Continue conversation if not satisfied


# Step 4: Graph Structure with Reflection and Satisfaction Check
def should_continue(state: SalesState):
    """Reflection-based stopping condition with satisfaction check."""
    latest_message = state['messages'][-1].content.lower()

    # Check if the user is satisfied based on reflection or conversation history
    if "awaiting for customer to response" in latest_message or  "end the conversation" in latest_message or "place an order" in latest_message or "i'm satisfied" in latest_message:
        print("User is satisfied. Ending the conversation.")
        return END

    if len(state["messages"]) > 12:
        print("Reached message limit, stopping.")
        return END

    return "reflection"  # Reflect after generation

class SalesGPT(BaseModel):
    """Controller model for the Sales Agent."""

    conversation_history: List[str] = []
    current_conversation_stage: str = "1"
    stage_analyzer_chain: StageAnalyzerChain = Field(...)
    sales_conversation_utterance_chain: SalesConversationChain = Field(...)

    salesperson_name: str = "Ted Lasso"
    company_name: str = "Sleep Haven"
    company_business: str = "Premium mattress and sleep solutions"

    @classmethod
    def from_llm(cls, llm, verbose: bool = False, **kwargs) -> "SalesGPT":
        """Initialize the SalesGPT Controller."""
        stage_analyzer_chain = StageAnalyzerChain.from_llm(llm, verbose=verbose)
        sales_conversation_utterance_chain = SalesConversationChain.from_llm(llm, verbose=verbose)
        return cls(
            stage_analyzer_chain=stage_analyzer_chain,
            sales_conversation_utterance_chain=sales_conversation_utterance_chain,
            **kwargs
        )

    def create_graph_state(self) -> SalesState:
        """Create the initial state for the graph."""
        return SalesState(
            messages=[HumanMessage(content=msg) if "User:" in msg else AIMessage(content=msg) for msg in self.conversation_history],
            conversation_stage=self.current_conversation_stage,
        )

    def retrieve_conversation_stage_from_graph(self, stage_id: str) -> str:
        """Retrieve the conversation stage based on stage ID from the graph."""
        conversation_stage_dict = {
            "1": "Introduction",
            "2": "Qualification",
            "3": "Value proposition",
            "4": "Needs analysis",
            "5": "Solution presentation",
            "6": "Objection handling",
            "7": "Close",
        }
        return conversation_stage_dict.get(stage_id, "1")

    async def run_graph(self, user_input: str):
        """Run the graph processing with user input."""
        state = self.create_graph_state()
        human_message = HumanMessage(content=f"User: {user_input} <END_OF_TURN>")
        state['messages'].append(human_message)

        # Process the graph asynchronously
        async for event in graph.astream(state):
            print(event)
            print("---")
        self.conversation_history.append(f"User: {user_input} <END_OF_TURN>")

    def execute_graph_step(self, user_input: str):
        """Run the graph step with human input."""
        if is_event_loop_running():
            # Directly await the run_graph method if an event loop is running
            return self.run_graph(user_input)
        else:
            # Use asyncio.run for other environments
            asyncio.run(self.run_graph(user_input))



In [147]:
class JapanEconomyDataChain(LLMChain):
    """Chain to gather macroeconomic data about Japan."""

    @classmethod
    def from_llm(cls, llm, verbose: bool = True) -> LLMChain:
        economic_data_prompt = """You are an economic researcher analyzing the current state of Japan's economy.
        Your task is to retrieve macroeconomic data for Japan, including GDP, inflation, unemployment, and trade balances.
        Analyze the retrieved data and provide a summary of Japan's current economic state."""

        prompt = PromptTemplate(
            template=economic_data_prompt,
            input_variables=["macro_data"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

class JapanRegionAnalysisChain(LLMChain):
    """Chain to analyze Japan's regions based on economic data."""

    @classmethod
    def from_llm(cls, llm, verbose: bool = True) -> LLMChain:
        region_analysis_prompt = """You are analyzing specific regions in Japan based on the available economic data.
        Use the macroeconomic data you have gathered to analyze how different regions of Japan are performing economically.
        Focus on key areas such as industrial output, consumer spending, and employment trends."""

        prompt = PromptTemplate(
            template=region_analysis_prompt,
            input_variables=["region_data"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)


In [148]:
from langchain_community.tools import TavilySearchResults

# Initialize the tool for researching Japan's economy
tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False,
)

async def gather_macro_data(state: SalesState) -> SalesState:
    """Use TavilySearchResults to gather macroeconomic data about Japan."""
    query = "current Japan macroeconomic data GDP inflation unemployment trade balances"

    # Use the tool to search for macroeconomic data (tool_input should be the query)
    macro_data = tool.run(tool_input=query)

    # Add the gathered data to the conversation state
    state['messages'].append(HumanMessage(content=f"Gathered macroeconomic data: {macro_data}"))

    state['macro_data'].append(AIMessage(content=macro_data))


    return state


async def japan_region_analysis(state: SalesState) -> SalesState:
    """Analyze specific regions in Japan based on the available economic data."""

    # Get macroeconomic data from the state (this was previously gathered in gather_macro_data)
    macro_data = state.get("macro_data", None)

    if macro_data is None:
        raise ValueError("Macro data not found in the state. Ensure gather_macro_data has been run first.")

    # Create the region analysis prompt
    region_analysis_prompt = f"""
    You are analyzing specific regions in Japan based on the available economic data.
    Use the following macroeconomic data to analyze how different regions of Japan are performing economically:

    {macro_data}

    Focus on key areas such as industrial output, consumer spending, and employment trends.
    """

    # Generate region analysis based on the macroeconomic data using the LLM (assuming llm is available globally or passed)
    region_analysis = await llm.generate([region_analysis_prompt])

    # Append the region analysis results to the state
    state['messages'].append(AIMessage(content=f"Region analysis result: {region_analysis}"))
    state['region_analysis_data'] = region_analysis  # Store the result in the state under 'region_analysis'

    return state


In [152]:
# Correctly define the conditional edge function
def should_continue(state: SalesState):
    """Reflection-based stopping condition with satisfaction check."""
    latest_message = state['messages'][-1].content.lower()

    # Check if the user is satisfied based on reflection or conversation history
    if "awaiting for customer to response" in latest_message or "end the conversation" in latest_message or "place an order" in latest_message or "i'm satisfied" in latest_message:
        print("User is satisfied. Ending the conversation.")
        return END

    if len(state["messages"]) > 12:
        print("Reached message limit, stopping.")
        return END

    return "reflection"  # Reflect after generation

# Example async function for staffing analysis node
async def staffing_analysis_node(state: SalesState) -> SalesState:
    """Node to evaluate the staffing needs and make actionable recommendations."""

    # Use the gathered macro data and region analysis to determine staffing requirements
    macro_data = state.get("macro_data")
    region_data = state.get("region_analysis")

    # Create a staffing analysis prompt
    staffing_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You are an expert in sales operations. Evaluate the current staffing needs based on the macroeconomic data and regional analysis for Japan.
            Suggest if additional staff are required, where to allocate existing staff, and whether certain processes could be automated to increase efficiency.
            Assess the gaps in staff skills and provide actionable recommendations to optimize staffing based on the current sales strategies."""),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    # Bind the state messages to the staffing prompt
    staffing_reflect = staffing_prompt | llm

    # Generate staffing recommendations based on the current sales strategy
    staffing_reflection = await staffing_reflect.ainvoke(state['messages'])

    # Output the reflection for debugging or logging purposes
    print("Staffing Analysis:", staffing_reflection.content)

    # Append the staffing reflection as an AIMessage to the conversation history
    state['messages'].append(AIMessage(content=staffing_reflection.content))

    state['staffing_reflection'].append(AIMessage(content=staffing_reflection.content))

    # Return the updated state for further decision-making
    return state


async def customer_market_segment_analysis_node(state: SalesState) -> SalesState:
    """Node to analyze customer market segments and provide actionable recommendations."""

    # Retrieve the gathered macro data and region analysis from the state
    macro_data = state.get("macro_data")
    region_data = state.get("region_analysis")

    # Create a customer market segment analysis prompt
    customer_segment_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You are an expert in market segmentation. Based on the macroeconomic data and regional analysis of Japan, evaluate the key customer segments.
            Identify high-potential customer segments, analyze their buying behavior, and suggest strategies to target them effectively.
            Also, assess how different economic trends affect customer preferences and market demand in these segments."""),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    # Bind the state messages to the customer segment prompt
    customer_segment_reflect = customer_segment_prompt | llm

    # Generate customer segment recommendations based on the macro and regional data
    customer_segment_reflection = await customer_segment_reflect.ainvoke(state['messages'])

    # Output the reflection for debugging or logging purposes
    print("Customer Segment Analysis:", customer_segment_reflection.content)

    # Append the customer segment reflection as an AIMessage to the conversation history
    state['messages'].append(AIMessage(content=customer_segment_reflection.content))

    # Append the analysis to a specific key in the state (similar to how staffing_reflection is handled)
    state['customer_segment_reflection'] = state.get('customer_segment_reflection', [])
    state['customer_segment_reflection'].append(AIMessage(content=customer_segment_reflection.content))

    # Return the updated state for further decision-making
    return state


# Define the state graph
builder1 = StateGraph(SalesState)
# Add nodes
builder1.add_node("gather_macro_data", gather_macro_data)
builder1.add_node("region_analysis", japan_region_analysis)
builder1.add_node("staffing_analysis", staffing_analysis_node)
builder1.add_node("target_customer_analysis", customer_market_segment_analysis_node)
builder1.add_node("reflection", reflection_node)

# Set up the flow
builder1.add_edge(START, "gather_macro_data")  # Start with gathering macro data

# Define conditional edges with correct syntax
builder1.add_conditional_edges("gather_macro_data", should_continue)  # This checks if we should continue to reflection or end

# Define the flow after reflection and satisfaction check
builder1.add_edge("reflection", "staffing_analysis")
builder1.add_edge("staffing_analysis", "target_customer_analysis")

# Compile the graph
graph = builder1.compile()


In [153]:

class JapanEconomyAgent(BaseModel):
    """Controller model for the Japan Economic Research Agent."""

    conversation_history: List[str] = []
    current_conversation_stage: str = "1"
    japan_economy_data_chain: JapanEconomyDataChain = Field(...)
    region_analysis_chain: JapanRegionAnalysisChain = Field(...)

    researcher_name: str = "Economic Researcher"

    @classmethod
    def from_llm(cls, llm, verbose: bool = False, **kwargs) -> "JapanEconomyAgent":
        """Initialize the Economic Research Agent."""
        japan_economy_data_chain = JapanEconomyDataChain.from_llm(llm, verbose=verbose)
        region_analysis_chain = JapanRegionAnalysisChain.from_llm(llm, verbose=verbose)
        return cls(
            japan_economy_data_chain=japan_economy_data_chain,
            region_analysis_chain=region_analysis_chain,
            **kwargs
        )

    def create_graph_state(self) -> SalesState:
      """Create the initial state for the graph."""
      # Initialize the messages with the conversation history, treating user and AI messages differently
      return SalesState(
          messages=[HumanMessage(content=msg) if "User:" in msg else AIMessage(content=msg) for msg in self.conversation_history],
          conversation_stage=self.current_conversation_stage,
          # Add the missing keys with default or provided values
          company_business="Default business",  # Replace with actual company business info if available
          company_name="Default company",  # Replace with actual company name if available
          salesperson_name="Default salesperson",  # Replace with actual salesperson name if available
          conversation_history=self.conversation_history  # Add conversation history if needed
      )
    async def run_graph(self, user_input: str):
        """Run the graph processing with user input."""
        # Create the initial state with the conversation history
        state = self.create_graph_state()

        # Add the user's message to the conversation history
        human_message = HumanMessage(content=f"User: {user_input} <END_OF_TURN>")
        state['messages'].append(human_message)
        #state.messages.append(human_message)  # Accessing like a class

        # Process the graph asynchronously
        async for event in graph.astream(state):
            print(event)
            print("---")

        # Append the user's input to the conversation history
        self.conversation_history.append(f"User: {user_input} <END_OF_TURN>")

    def execute_graph_step(self, user_input: str):
        """Run the graph step with human input."""
        if is_event_loop_running():
            # Directly await the run_graph method if an event loop is running
            return self.run_graph(user_input)
        else:
            # Use asyncio.run for other environments
            asyncio.run(self.run_graph(user_input))


In [154]:
import asyncio

# Step 1: Initialize the agent with the model and chains
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=32768
)

# Initialize the agent
japan_economy_agent = JapanEconomyAgent.from_llm(llm)

# Step 2: Define a function to run the agent
async def run_economy_agent(user_input: str):
    """Function to run the agent and simulate user input."""
    await japan_economy_agent.run_graph(user_input)

# Step 3: Run the agent with user input
user_input = "Please gather the latest macroeconomic data about Japan and suggest sales strategies."

# Check if the event loop is running
if is_event_loop_running():
    await run_economy_agent(user_input)
else:
    asyncio.run(run_economy_agent(user_input))


{'gather_macro_data': {'messages': [HumanMessage(content='User: Please gather the latest macroeconomic data about Japan and suggest sales strategies. <END_OF_TURN>', additional_kwargs={}, response_metadata={}, id='e582b961-3c71-4054-b94a-7be0ad4220c2'), HumanMessage(content='Gathered macroeconomic data: [{\'url\': \'https://www.statista.com/topics/11889/key-economic-indicators-of-japan/\', \'content\': \'Discover all statistics and data on Key economic indicators of Japan now on statista.com! ... Current account balance as share of GDP Japan 1982-2029 ... Premium Statistic Trade balance of goods ...\'}, {\'url\': \'https://tradingeconomics.com/japan/balance-of-trade\', \'content\': \'Balance of Trade in Japan averaged 277.64 JPY Billion from 1963 until 2024, reaching an all time high of 1608.68 JPY Billion in September of 2007 and a record low of -3506.43 JPY Billion in January of 2023. This page provides - Japan Balance of Trade - actual values, historical data, forecast, chart, stati